# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_df = pd.read_csv('cities.csv')
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


# Store 'Lat' and 'Lng' into  locations
locations = cities_df[['Lat', 'Lng']]

#Humidity as the weight
humidity = cities_df["Humidity"].astype(float)

#Heatmap layer to map
fig = gmaps.figure(zoom_level=10, center= (4,-72))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = cities_df.loc[(cities_df['Max Temp'] >=80)&(cities_df['Wind Speed']<10)\
                              &(cities_df['Cloudiness']>=50), :].dropna(how='any')
ideal_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,23,maragogi,97,BR,1558378610,69,-9.01,-35.22,83.94,9.42
45,45,rosita,72,NI,1558378762,71,13.92,-84.40,84.12,4.41
49,49,garchitorena,59,PH,1558378763,75,13.88,123.70,84.30,6.11
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88
72,72,butaritari,55,KI,1558378767,77,3.07,172.79,83.94,9.13
108,108,bonthe,63,SL,1558378774,70,7.53,-12.50,85.38,9.78
111,111,port blair,99,IN,1558378774,71,11.67,92.75,85.20,7.18
130,130,cravo norte,100,CO,1558378778,67,6.30,-70.20,85.02,3.09
147,147,nagua,75,DO,1558378782,70,19.37,-69.85,89.60,6.93
158,158,trat,93,TH,1558378784,70,12.25,102.51,91.00,1.88


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df & add a "Hotel Name" column to the DataFrame.
hotel_df = ideal_weather
hotel_df ["Hotel Name"] = ""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
23,23,maragogi,97,BR,1558378610,69,-9.01,-35.22,83.94,9.42,
45,45,rosita,72,NI,1558378762,71,13.92,-84.40,84.12,4.41,
49,49,garchitorena,59,PH,1558378763,75,13.88,123.70,84.30,6.11,
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,
72,72,butaritari,55,KI,1558378767,77,3.07,172.79,83.94,9.13,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

Retrieving Results for Index 23: maragogi.
Closest hotel in maragogi is Pousada Camurim Grande.
Retrieving Results for Index 45: rosita.
Closest hotel in rosita is Hotel Rosita.
Retrieving Results for Index 49: garchitorena.
Missing field/result... skipping.
Retrieving Results for Index 52: taybad.
Closest hotel in taybad is دفتردستک محمدسیاه.
Retrieving Results for Index 72: butaritari.
Missing field/result... skipping.
Retrieving Results for Index 108: bonthe.
Closest hotel in bonthe is Bonthe Holiday Village.
Retrieving Results for Index 111: port blair.
Closest hotel in port blair is SeaShell, Port Blair.
Retrieving Results for Index 130: cravo norte.
Closest hotel in cravo norte is Hotel El Merecure.
Retrieving Results for Index 147: nagua.
Closest hotel in nagua is Hotel Juliana.
Retrieving Results for Index 158: trat.
Closest hotel in trat is Avada Hotel.
Retrieving Results for Index 172: antsohihy.
Closest hotel in antsohihy is Hotel de Ville.
Retrieving Results for Index 173: 

In [8]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
23,23,maragogi,97,BR,1558378610,69,-9.01,-35.22,83.94,9.42,Pousada Camurim Grande
45,45,rosita,72,NI,1558378762,71,13.92,-84.40,84.12,4.41,Hotel Rosita
49,49,garchitorena,59,PH,1558378763,75,13.88,123.70,84.30,6.11,
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,دفتردستک محمدسیاه
72,72,butaritari,55,KI,1558378767,77,3.07,172.79,83.94,9.13,
108,108,bonthe,63,SL,1558378774,70,7.53,-12.50,85.38,9.78,Bonthe Holiday Village
111,111,port blair,99,IN,1558378774,71,11.67,92.75,85.20,7.18,J Hotel
130,130,cravo norte,100,CO,1558378778,67,6.30,-70.20,85.02,3.09,Hotel El Merecure
147,147,nagua,75,DO,1558378782,70,19.37,-69.85,89.60,6.93,Hotel Juliana
158,158,trat,93,TH,1558378784,70,12.25,102.51,91.00,1.88,Avada Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Pousada Camurim Grande</dd>\n<dt>City</dt><dd>maragogi</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Rosita</dd>\n<dt>City</dt><dd>rosita</dd>\n<dt>Country</dt><dd>NI</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>garchitorena</dd>\n<dt>Country</dt><dd>PH</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>دفتردستک محمدسیاه</dd>\n<dt>City</dt><dd>taybad</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>butaritari</dd>\n<dt>Country</dt><dd>KI</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Bonthe Holiday Village</dd>\n<dt>City</dt><dd>bonthe</dd>\n<dt>Country</dt><dd>SL</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>SeaShell, Port Blair</dd>\n<dt>City</dt><dd>port blair</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel El Merecure</dd>\n<dt>City</dt><dd>cravo norte</dd>\n<dt>Country</dt><dd>CO</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Juliana</dd>

In [ ]:
# Add marker layer ontop of heat map
fig=gmaps.figure()
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(marker_layer)

# Display figure
fig